### Vanilla CTR Zeroshot

In [1]:
import pickle
import pandas as pd
import numpy as np
import os
import time
from dotenv import load_dotenv
import re

In [81]:
inference_path = 'ctr_vanilla_test_inference_mixtral.pkl'
with open(inference_path, 'rb') as f:
    inference_dict = pickle.load(f)
print(len(inference_dict))

6040


In [3]:
ratings_df = pd.read_csv('../ratings.csv')
ratings_df.head()

,user_id,movie_id,rating
0,1,3186,4
1,1,1721,4
2,1,1022,5
3,1,1270,5
4,1,2340,3


In [24]:
user_valid_movie_rating_df = ratings_df.groupby('user_id').nth(-2)
user_valid_movie_rating_df.head()

,user_id,movie_id,rating
51,1,1566,4
452,10,2693,5
528,100,1208,2
612,1000,3363,5
989,1001,3756,2


In [86]:
pattern = r"Prediction:(?:\s+)?(Yes|No)\b"
invalid_users = []
cnt = cntInvalid = cntY = cntN = 0
user_inference_dict = dict()
for user, inference in inference_dict.items():
    cnt += 1
    # print(user, inference)
    # Use re.findall to find all matches in the string
    matches = re.findall(pattern, inference, re.IGNORECASE)
    if len(matches) == 0:
        cntInvalid += 1
        invalid_users.append(user)
        user_inference_dict[user] = 0
    for match in matches:
        if match == 'Yes':
            cntY += 1
            user_inference_dict[user] = 1
            break
        elif match == 'No':
            cntN += 1
            user_inference_dict[user] = 0
            break
    # print('*'*100)
    # if cnt == 10:
    #     break
print(f"Yes: {cntY} -- No: {cntN} -- Invalid: {cntInvalid}")
print("invalid_users:", invalid_users)
print(len(user_inference_dict))

Yes: 5549 -- No: 111 -- Invalid: 380
invalid_users: [1031, 1039, 1046, 1070, 1091, 1123, 1126, 1154, 1157, 1171, 1189, 1213, 1236, 1250, 1282, 130, 1305, 1310, 1319, 1336, 1363, 1369, 1406, 1412, 1413, 1416, 1429, 1444, 1452, 1453, 1459, 1471, 1488, 15, 1518, 1531, 1534, 1539, 1541, 1549, 1598, 160, 1604, 1612, 1616, 1621, 1640, 1651, 1652, 1653, 1655, 1656, 1664, 1668, 1674, 168, 1703, 1708, 171, 1711, 1713, 1730, 1751, 1754, 1763, 1771, 1783, 1787, 1813, 1818, 1864, 1886, 1893, 1898, 1906, 1967, 1989, 1991, 1993, 20, 2006, 2045, 2066, 2080, 2086, 2116, 2128, 2131, 2152, 217, 2171, 2182, 2193, 2207, 2210, 226, 2268, 2287, 2295, 2311, 2347, 2357, 2363, 2371, 2379, 2385, 2399, 2441, 2443, 2450, 2470, 2490, 250, 2511, 2516, 2530, 2542, 2549, 2553, 2563, 2613, 2640, 2653, 2673, 2686, 2690, 2696, 2714, 2729, 2735, 2759, 276, 2763, 2799, 2816, 2819, 2834, 2836, 2884, 2893, 2908, 2920, 2956, 2962, 2977, 2982, 2999, 3009, 3019, 3021, 304, 3040, 305, 3061, 3071, 3076, 3122, 3124, 3165, 3168, 3

In [87]:
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, precision_score, recall_score
y_true = []
y_pred = []

fp_users = []
fn_users = []

for user, inference in user_inference_dict.items():
    original_rating = user_valid_movie_rating_df[user_valid_movie_rating_df['user_id'] == user]['rating'].values[0]
    # print(original_rating)
    if original_rating >= 3:
        y_true.append(1)
        if inference == 0:
            fn_users.append(user)
    else:
        y_true.append(0)
        if inference == 1:
            fp_users.append(user)
    y_pred.append(inference)

conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

auc_roc = roc_auc_score(y_true, y_pred)
print("AUC:", auc_roc)

precision = precision_score(y_true, y_pred)
print("Precision:", precision)

recall = recall_score(y_true, y_pred)
print("Recall:", recall)

accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

Confusion Matrix:
[[ 118  935]
 [ 373 4614]]
AUC: 0.5186331565584289
Precision: 0.831501171382231
Recall: 0.9252055343894124
Accuracy: 0.7834437086092715


In [31]:
movies_df = pd.read_csv('../movie_summary_25_words.csv')
movies_df[movies_df.movie_id == 1466]

,movie_id,name,cleaned_genre,year,summary
1439,1466,Donnie Brasco,"Crime,Drama",1997,"""Donnie Brasco"" is a gripping crime drama base..."


In [9]:
ctr_valid_dataset_path = './ctr_zeroshot_dataset/ctr_valid_zeroshot_dataset.pkl'
with open(ctr_valid_dataset_path, 'rb') as f:
    ctr_valid_data_dict = pickle.load(f)
print(len(ctr_valid_data_dict))

6040


In [14]:
print(ctr_valid_data_dict[1005])


    You are an expert movie critic. You are provided with the user profile and list of recent movies that the user has watched and whether the user likes it or not.
    User Profile -
    The user is a male lawyer between 35 to 44 years of age
            The user has a diverse taste in movies, with a preference for those that are thought-provoking and emotionally impactful. They enjoy movies that explore complex themes and blur the lines between reality and fantasy, as seen in their appreciation for "Being John Malkovich" and "Bowfinger." The user also has a soft spot for classic Christmas films, as evidenced by their fondness for "Miracle on 34th Street."
            
            In terms of genres, the user leans towards drama and comedy, with a particular interest in coming-of-age stories and character-driven narratives. They are drawn to movies that feature strong performances and explore the complexities of human relationships, as seen in their appreciation for "Diner" and "Schi

In [14]:
for user in invalid_users:
    print(user, inference_dict[user])

1031 
    Explanation:
    Based on the user's preference for science fiction movies, Gabbeh, an Iranian film, is unlikely to
1039 

To predict whether the user will like or dislike the next movie, we need to analyze the user's movie watching pattern and their preferences. Based
1046 
































1070 
































1091 
































1123 
































1126 
































1154 
    Explanation:
    Based on the user's preference for coming-of-age stories and her dislike for romantic dramas, it
1157 

Answer: Yes

Explanation: The user is a male executive/managerial between 45 to 49 years of age
1171 
































1189 
































1213 

To predict whether the user will like or dislike the next movie, we can analyze the user's movie preferences based on the given data. Here
1236 

Based on the user's viewing history, it appears that she enjoys action-pac